In [1]:
import plotly.express as px
import pandas as pd
import polars as pl
import numpy as np
from math import ceil, log2

In [88]:
df:pl.DataFrame = pl.read_csv("/home/michael/Datasets/temp/script_job_df50654add33025f3714f49a6e3dd020_0.csv")
df.describe()

statistic,Cnt,Stake
str,f64,f64
"""count""",13346.0,13346.0
"""null_count""",0.0,0.0
"""mean""",307.875993,48237.086765
"""std""",6133.584392,303626.368438
"""min""",1.0,0.4835
"""25%""",1.0,3285.0
"""50%""",2.0,7112.0
"""75%""",11.0,15426.0
"""max""",370419.0,1.352925e7


In [89]:

total = int(df['Cnt'].sum())

df =  (
        df
        .filter([pl.col('Stake')>0])
        .with_columns([
                pl.col('Stake').cast(pl.Int32).cast(pl.String).cast(pl.Categorical).alias('Stk'),
                (pl.col('Cnt')/total).round(6).alias('Per')
        ])
)

v1 = df.to_pandas()
v1

,Cnt,Stake,Stk,Per
0,2,4.835000e-01,0,0.000000
1,5327,1.000000e+00,1,0.001296
2,5416,2.000000e+00,2,0.001318
3,81342,3.000000e+00,3,0.019796
4,27502,4.000000e+00,4,0.006693
...,...,...,...,...
13341,1,7.628000e+06,7628000,0.000000
13342,1,8.000000e+06,8000000,0.000000
13343,2,8.823529e+06,8823529,0.000000
13344,1,9.000000e+06,9000000,0.000000


In [87]:
df = df.filter([pl.col('Per')>0.001]).select(pl.all())
left = int(df['Cnt'].sum())
print(f"Data coverage==>{np.round(left/total,2)},{len(df)}")

Data coverage==>0.8,105


In [77]:
df.describe()

statistic,Cnt,Stake,Stk,Per
str,f64,f64,str,f64
"""count""",105.0,105.0,"""105""",105.0
"""null_count""",0.0,0.0,"""0""",0.0
"""mean""",31225.942857,455.628571,null,0.0076
"""std""",62000.899768,1329.225847,null,0.015089
"""min""",4125.0,1.0,null,0.001004
"""25%""",5557.0,27.0,null,0.001352
"""50%""",7806.0,59.0,null,0.0019
"""75%""",20301.0,180.0,null,0.004941
"""max""",370419.0,10000.0,null,0.09015


In [78]:
fig = px.bar(df.to_pandas(), x='Stk',y='Cnt')  # Replace 20 with your calculated number of bins
fig.show()

In [39]:
def calculate_bins(data, column):
    """
    Calculate the number of bins using the Freedman-Diaconis rule.

    Parameters:
    - data: pandas.DataFrame
    - column: str, the column name for which to calculate the number of bins

    Returns:
    - int, the suggested number of bins
    """
    # Extract the relevant data
    col_data = data[column].dropna()  # Remove NaN values for accurate calculations

    # Calculate the interquartile range (IQR)
    Q1 = np.percentile(col_data, 25)
    Q3 = np.percentile(col_data, 75)
    IQR = Q3 - Q1

    # Calculate bin width using the Freedman-Diaconis rule
    # Avoid division by zero by handling the zero-IQR case
    if IQR == 0:
        bin_width = np.ptp(col_data) / len(col_data)  # Peak-to-peak range divided by the number of observations
    else:
        bin_width = 2 * IQR / (len(col_data) ** (1/3))

    # Calculate the number of bins
    n_bins = int(np.ceil((col_data.max() - col_data.min()) / bin_width))
    
    return n_bins

In [40]:
def calculate_bins_sturges(data, column):
    """
    Calculate the number of bins using Sturges' formula.

    Parameters:
    - data: pandas.DataFrame
    - column: str, the column name for which to calculate the number of bins

    Returns:
    - int, the suggested number of bins
    """
    # Number of observations
    n = len(data[column].dropna())  # Drop NaN to avoid counting them

    # Apply Sturges' formula
    k = ceil(log2(n) + 1)

    return k

In [41]:
n_bins_x = calculate_bins_sturges(df.to_pandas(), 'Stake')
n_bins_x

8

In [73]:
fig = px.histogram(df.to_pandas(), x='Stake', nbins=n_bins_x)  # Replace 20 with your calculated number of bins
fig.show()


In [43]:
n_bins_x = calculate_bins(df.to_pandas(), 'Stake')
n_bins_x

155

In [45]:

def calculate_bins_sturges(n):
    return ceil(log2(n) + 1)

# Calculate the number of bins for the 'Stake' column
n_bins = calculate_bins_sturges(len(df.to_pandas()['Stake'].dropna()))

# Create the histogram using Plotly
fig = px.histogram(df.to_pandas(), x='Stake', nbins=n_bins, title="Histogram of Stake")
fig.show()